In [228]:
import pandas as pd
import numpy as np
from pypfopt import black_litterman
from pypfopt import BlackLittermanModel
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')


In [229]:
"""
Asset info
"""

asset_info = {
    'h20025.CSI': (5669.2288, 4705.2669),
    '399244.SZ': (640.136, 600.1125),
    '931646CNY01.CSI': (1344.0437, 1247.5575),
    'h40006.SH': (5912.7329, 6335.0402),
    '983087.CNI': (4835.5539, 4471.1056)
}

index_list = list(asset_info.keys())

In [230]:
"""
BL Table
"""

# P matrix cannot be orthogonalised
bl_table_non_orthogonal = ['0', '0', '0', '1', '-1', '0.1468', 
                           '-1', '1', '0', '0', '0', '0.1075', 
                           '0', '0', '1', '0', '0', '-0.0722', 
                           '0', '0', '0', '1', '0', '0.0715', 
                           '0', '0', '-1', '1', '0', '0.1437', ]

# P matrix can be orthogonalised
bl_table_orthogonal = ['1', '0', '0', '0', '0', '-0.17', 
                       '0', '1', '-1', '0', '0', '0.00928', 
                       '0', '-1', '0', '1', '0', '0.1339468', 
                       '-1', '0', '0', '1', '0', '0.24145741', 
                       '0', '0', '0', '1', '-1', '0.14679184', ]

In [231]:
"""
Data Import
"""

asset_index = pd.read_csv("data/AIDX.csv", encoding='gbk')

index_min_weight = [0, 0, 0, 0, 0]
index_max_weight = [1, 1, 1, 1, 1]
weight_constrains = list(zip(index_min_weight, index_max_weight))
backtest_day = 30

asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
asset_index.sort_values(by='TRADE_DT', inplace=True)
asset_index.set_index('TRADE_DT', inplace=True)
asset_index = asset_index.loc[:"20230901"]
asset_index = asset_index.pivot(columns='S_IRDCODE', values='CLOSE').ffill()[index_list]

tmp_close = asset_index.tail(backtest_day)

In [232]:
"""
BL table (Auto Generating)
"""

def generate_bl_matrix(asset_info):
    views = {asset: (final - initial) / initial for asset, (initial, final) in asset_info.items()}
    P_orthogonal = np.eye(len(views)) # individual return / orthogonal
    Q = np.array(list(views.values())).ravel() # 1D array
    P_non_orthogonal = np.full((len(views), len(views)), 0.5) #  Example where each view partially affects each asset
    
    return P_orthogonal, P_non_orthogonal, Q

P_orthogonal_actual, P_random, Q_actual = generate_bl_matrix(asset_info)
print(f"Actual return")
print(Q_actual)


Actual return
[-0.09763079 -0.0632274   0.61045618  0.06072781  0.06837308]


In [233]:
"""
BL table (Manual Modification)
"""

def transform_bl_table(bl_table):
    bl_table = np.array(bl_table)
    bl_table = bl_table.reshape(-1,(len(index_list)+1))
    bl_table = pd.DataFrame(bl_table,columns=index_list+['Q'])
    bl_table = bl_table.replace('',np.nan)
    bl_table = bl_table.astype(float)
    bl_table = bl_table.reset_index(drop=True)
    bl_table.index += 1
    bl_table = bl_table.fillna(0)

    P = np.array(bl_table.iloc[:,:-1])
    Q =  np.array(bl_table.iloc[:,-1])
    
    return P, Q


def gram_schmidt(V):
    U = np.copy(V).astype(np.float64)
    for i in range(1, V.shape[0]):
        for j in range(i):
            U[i] -= np.dot(U[j], U[i]) / np.dot(U[j], U[j]) * U[j]
    for i in range(V.shape[0]):
        U[i] /= np.linalg.norm(U[i])
    return U

P, Q = transform_bl_table(bl_table_non_orthogonal)
print(f"Non-orthogonal test")
print(gram_schmidt(P))

Non-orthogonal test
[[ 0.         -0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.70710678  0.70710678  0.          0.        ]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.70710678  0.70710678]
 [        nan         nan         nan         nan         nan]]


In [234]:
"""
Orthogonalisation
"""
P_can_orthogonal, Q_can_orthogonal = transform_bl_table(bl_table_orthogonal)
print(f"Orthogonal test")
print(gram_schmidt(P_can_orthogonal))

def orthogonalise(P_can_orthogonal, Q_actual):
    P_orthogonal = gram_schmidt(P_can_orthogonal)
    Q_orthogonal = np.array([np.dot(row, Q_actual) for row in P_orthogonal])
    return P_orthogonal, Q_orthogonal


P_orthogonal, Q_orthogonal = orthogonalise(P_can_orthogonal, Q_actual)
print(f"Orthogonalisation Q")
print(Q_orthogonal)

Orthogonal test
[[ 0.          0.          1.          0.          0.        ]
 [ 0.         -0.70710678  0.          0.70710678  0.        ]
 [-0.70710678  0.          0.          0.          0.70710678]
 [-0.5         0.5         0.          0.5        -0.5       ]
 [-0.5        -0.5         0.         -0.5        -0.5       ]]
Orthogonalisation Q
[0.61045618 0.08764957 0.11738246 0.01337906 0.01587865]


In [235]:
"""
Run BL
"""

S = (tmp_close.pct_change().dropna()).cov()
mcaps = {i:1 for i in list(S.index)}
delta = 1
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)

# Orthogonoal, Actual
bl_orthogonal_actual = BlackLittermanModel(S, pi=market_prior, P=P_orthogonal_actual, Q=Q_actual)
ret_bl_orthogonal_actual = bl_orthogonal_actual.bl_returns()

# Non-orthogonal
bl_non_orthogonal = BlackLittermanModel(S, pi=market_prior, P=P, Q=Q)
ret_bl_non_orthogonal = bl_non_orthogonal.bl_returns()

# Orthogonal
bl_orthogonal = BlackLittermanModel(S, pi=market_prior, P=P_orthogonal, Q=Q_orthogonal)
ret_bl_orthogonal = bl_orthogonal.bl_returns()

# Can be orthogonalised
bl_can_orthogonal = BlackLittermanModel(S, pi=market_prior, P=P_can_orthogonal, Q=Q_can_orthogonal)
ret_bl_can_orthogonal = bl_can_orthogonal.bl_returns()

# Random
bl_random = BlackLittermanModel(S, pi=market_prior, P=P_random, Q=Q_actual)
ret_bl_random = bl_random.bl_returns()

In [236]:
"""
MSE
"""

mse = mean_squared_error(ret_bl_orthogonal_actual, Q_actual)
print(f"MSE, Orthogonal Actual: {mse}")

mse = mean_squared_error(ret_bl_non_orthogonal, Q_actual)
print(f"MSE, Non-Orthogonal: {mse}")

mse = mean_squared_error(ret_bl_orthogonal, Q_actual)
print(f"MSE, Orthogonal: {mse}")

mse = mean_squared_error(ret_bl_can_orthogonal, Q_actual)
print(f"MSE, Can be Orthogonalised: {mse}")

mse = mean_squared_error(ret_bl_random, Q_actual)
print(f"MSE, Random: {mse}")

MSE, Orthogonal Actual: 0.036098874325698374
MSE, Non-Orthogonal: 0.035511292708189604
MSE, Orthogonal: 0.036050441504895316
MSE, Can be Orthogonalised: 0.030851124761751926
MSE, Random: 0.07148721836557932


### Test 1 - 4Drop/1Rise

#### Asset Info
```
asset_info = {
    'h20025.CSI': (5669.2288, 4705.2669),
    '399244.SZ': (640.136, 600.1125),
    '931646CNY01.CSI': (1344.0437, 1247.5575),
    'h40006.SH': (5912.7329, 6335.0402),
    '983087.CNI': (4835.5539, 4471.1056)
}
```

Actual return
[-0.17003404 -0.06252343 -0.07178799  0.07142337 -0.07536847]

#### BL Table
```
# P matrix cannot be orthogonalised
bl_table_non_orthogonal = ['0', '0', '0', '1', '-1', '0.1468', 
                           '-1', '1', '0', '0', '0', '0.1075', 
                           '0', '0', '1', '0', '0', '-0.0722', 
                           '0', '0', '0', '1', '0', '0.0715', 
                           '0', '0', '-1', '1', '0', '0.1437', ]

# P matrix can be orthogonalised
bl_table_orthogonal = ['1', '0', '0', '0', '0', '-0.17', 
                       '0', '1', '-1', '0', '0', '0.00928', 
                       '0', '-1', '0', '1', '0', '0.1339468', 
                       '-1', '0', '0', '1', '0', '0.24145741', 
                       '0', '0', '0', '1', '-1', '0.14679184', ]
```

#### Result

MSE, Orthogonal Actual: 0.0024613589538743236

MSE, Non-Orthogonal: 0.005495074009677869

MSE, Orthogonal: 0.002138758149810064

MSE, Can be Orthogonalised: 0.0015669760068734353

MSE, Random: 0.006543571463257819

### Test 2 - 2Drop/3Rise

#### Asset Info
```
asset_info = {
    'h20220.CSI': (5723.7547, 5164.94),
    '399419.SZ': (1806.114, 1691.9181),
    'h20726.CSI': (5853, 9426),
    'CN2394.CNI': (10848.6473, 11507.4619),
    '000037.SH': (6334.1099, 6767.1925)
}
```

Actual return
[-0.09763079 -0.0632274   0.61045618  0.06072781  0.06837308]

#### BL Table
```
# P matrix cannot be orthogonalised
bl_table_non_orthogonal = ['0', '-1', '1', '0', '0', '0.67368358', 
                           '0', '0', '1', '0', '0', '0.61045618', 
                           '-1', '1', '0', '0', '0', '0.03440339', 
                           '0', '0', '0', '-1', '1', '0.00764527', 
                           '1', '0', '0', '0', '0', '-0.09763079', ]

# P matrix can be orthogonalised
bl_table_orthogonal = ['0', '0', '1', '0', '0', '0.61045618', 
                       '0', '-1', '0', '1', '0', '0.12395521', 
                       '-1', '0', '0', '0', '1', '0.16600387', 
                       '-1', '0', '0', '1', '0', '0.1583586', 
                       '0', '0', '1', '-1', '0', '0.54972837', ]
```

#### Result

MSE, Orthogonal Actual: 0.036098874325698374

MSE, Non-Orthogonal: 0.035511292708189604

MSE, Orthogonal: 0.036050441504895316

MSE, Can be Orthogonalised: 0.030851124761751926

MSE, Random: 0.07148721836557932